In [ ]:
!apt-get update && apt-get install -y ffmpeg

In [ ]:
!pip3 install openai-whisper

In [ ]:
!pip3 install pytubefix

In [ ]:
import whisper
model = whisper.load_model("large")

In [ ]:
import pytubefix as pt
yt = pt.YouTube("https://www.youtube.com/watch?v=Z07Ewop7rQA")
stream = yt.streams.filter(only_audio=True)[0]
stream.download(filename="aip.mp3")

In [ ]:

result = model.transcribe("aip.mp3")

In [ ]:
import json
with open("aip.json", "w") as f:
    json.dump(result, f)

In [ ]:
list(result.keys())

In [ ]:
print(result['text'])

In [ ]:
!apt-get update && apt-get install ffmpeg -y

In [ ]:
!pip3 install moviepy openai python-dotenv pydub llama-index llama-index-llms-openai

In [ ]:
!ls /workspace/data

In [ ]:
from openai import OpenAI
from moviepy.editor import VideoFileClip
import os
from dotenv import load_dotenv
load_dotenv("/workspace/repos/agentic-ai/.env")

def video_to_audio(video_path, output_audio_path):
    """
    Convert a video to audio and save it to the output path.

    Parameters:
    video_path (str): The path to the video file.
    output_audio_path (str): The path to save the audio to.

    """
    clip = VideoFileClip(video_path)
    audio = clip.audio
    audio.write_audiofile(output_audio_path)



In [ ]:
video_to_audio("/workspace/data/video1512218125.mp4", "/workspace/data/video_audio.mp3")

In [ ]:
from pydub import AudioSegment

def split_audio_into_chunks(file_path, output_dir, max_chunk_size_mb=24):
    # Load the audio file
    audio = AudioSegment.from_mp3(file_path)
    
    # Calculate the maximum duration for each chunk based on the size limit
    max_chunk_size_bytes = max_chunk_size_mb * 1024 * 1024 * 10
    max_chunk_duration = (max_chunk_size_bytes / len(audio.raw_data)) * len(audio)
    
    # Split and export the audio file into chunks
    start_time = 0
    chunk_index = 1
    while start_time < len(audio):
        print(f"Exporting chunk {chunk_index}/{len(audio)/max_chunk_duration}")
        end_time = min(start_time + max_chunk_duration, len(audio))
        chunk = audio[start_time:end_time]
        chunk.export(f"{output_dir}/chunk_{chunk_index}.mp3", format="mp3")
        start_time = end_time
        chunk_index += 1

# Example usage
split_audio_into_chunks("/workspace/data/video_audio.mp3", "/workspace/data")


In [ ]:
from openai import OpenAI
import os

OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")
os.environ["OPENAI_API_KEY"] = OPENAI_API_KEY

client = OpenAI(api_key=OPENAI_API_KEY)

transcription = []
chunk_list = [chunk for chunk in os.listdir("/workspace/data") if chunk.startswith("chunk")]
chunk_list=sorted(chunk_list)
print(chunk_list)

for chunk in chunk_list:
      print(f"Transcribing {chunk}...")
      audio_file = open(f"/workspace/data/{chunk}", "rb")
      transcription.append(client.audio.transcriptions.create(
          model="whisper-1",
          file=audio_file
      ))
# print(transcription.text)

In [ ]:
import pickle
with open("/workspace/data/transcription.pkl", "wb") as f:
    pickle.dump(transcription, f)

In [ ]:
import pickle, os
from llama_index.core.response_synthesizers import TreeSummarize
from llama_index.core import (
    Settings,
)


with open("/workspace/data/transcription.pkl", "rb") as f:
    transcription = pickle.load(f)
summarizer = TreeSummarize(verbose=True)

In [ ]:
from llama_index.core import (
    Settings,
)
from llama_index.llms.openai import OpenAI as LOpenAI

llm = LOpenAI(model="gpt-4o", max_tokens=4000)
Settings.llm = llm

In [ ]:
prompt_summary = "You are a professional executive of AlphaTrAI. Your job is to summarize this text in great detail from a video transcription. The summary will be distributed to investors and stakeholders, so give a lot of details and examples from the transcription."
response = await summarizer.aget_response(prompt_summary, [doc.text for doc in transcription])


In [ ]:
full_doc = " ".join([doc.text for doc in transcription])

In [ ]:
prompt_summary = f"You are a professional executive at AlphaTrAI. Your job is to summarize the text from a video transcription. The summary will be a memo distributed to investors and stakeholders. Be sure it is factual, optimistic, and containing little information about change. The transcription is as follows:\n\n{full_doc}"


In [ ]:
response = llm.complete(prompt_summary, max_tokens=5000)

In [ ]:
print(response.text)

In [ ]:
# !cp '/workspace/repos/agentic-ai/PPM - MCG MADISON RIDGE DST.pdf' /workspace/data
# !cp '/workspace/repos/agentic-ai/MASTER - PYTHON - SCORING MODEL - MCG MADISON RIDGE DST - v2.0.xlsx' /workspace/data
# !apt update
# !apt install tmux vim -y
!pip3 install llama-index llama-parse llama-index-llms-huggingface llama-index-embeddings-huggingface llama-index-llms-ollama llama-index-finetuning
!pip3 install openpyxl sentencepiece protobuf evaluate rouge_score absl-py tensorboardX bitsandbytes peft accelerate python-dotenv graspologic
# !pip3 install flash-attn --no-build-isolation
# !curl -fsSL https://ollama.com/download/ollama-linux-amd64.tgz | tar zx -C /usr


!ollama pull llama3.1:latest

In [ ]:

import json
import os
from dotenv import load_dotenv
from llama_index.llms.ollama import Ollama
from llama_parse import LlamaParse
from llama_index.embeddings.huggingface import HuggingFaceEmbedding
from llama_index.core import (
    SimpleDirectoryReader,
    VectorStoreIndex,
    StorageContext,
    load_index_from_storage,
    PropertyGraphIndex,
    Settings,
    Document
)

import nest_asyncio
nest_asyncio.apply()

load_dotenv('/workspace/repos/agentic-ai/.env')
access_token = os.getenv('HF_TOKEN')
llama_api_key = os.getenv('LLAMA_API_KEY')

# model_name, num_ctx = "mistral-nemo", 128000
model_name, num_ctx = "llama3.1", 128000

addtion_kwargs = {"max_new_tokens": 3000}
system_prompt = "You are an expert in creating marketing materials for financial firms. You take corporate documents and turn them into marketing materials."
# system_prompt = "You are an expert in marketing for financial firms. You take corporate documents and turn them into marketing materials. You are working on a project for a financial firm that is looking to market their products to investors."
# llm = Ollama(model=model_name, url="http://127.0.0.1:11434", system_prompt=system_prompt, context_window=num_ctx, model_type="chat", is_function_calling_model=False)
llm = Ollama(model=model_name, url="http://127.0.0.1:11434", context_window=num_ctx, model_type="chat", is_function_calling_model=False, 
             request_timeout=1000.0, system_prompt=system_prompt, additional_kwargs=addtion_kwargs)
            #  request_timeout=1000.0, additional_kwargs=addtion_kwargs)
Settings.llm = llm
llm.metadata


Settings.chunk_size = 200
Settings.chunk_overlap = 50
embed_model_name = "Alibaba-NLP/gte-Qwen2-1.5B-instruct"
# embed_model_name = "BAAI/bge-small-en-v1.5"
# embed_model_name = "hkunlp/instructor-base"
print("loading embed model...")
embed_model = HuggingFaceEmbedding(model_name=embed_model_name, device="cuda")

Settings.embed_model = embed_model


!pip3 install llama-parse

In [ ]:
parser = LlamaParse(api_key=llama_api_key, result_type="markdown", split_by_page=False)
# pdf_urls = ["/workspace/data/AIP - Capital Raise Proposal.pdf", "/workspace/data/Access Pre-IPOs Presentation June 2024.pdf"] 
pdf_urls = ["/workspace/data/Access Pre-IPOs Presentation June 2024.pdf"] 
documents = []
for pdf in pdf_urls:
    print('processing pdf:', pdf)
    documents += parser.load_data(pdf)

with open("aip.json", "r") as f:
    result = json.load(f)
    documents.append(Document(text=result['text']))

persist_dir="/workspace/data/vector_index"
from llama_index.core import VectorStoreIndex
vector_index = VectorStoreIndex.from_documents(documents, 
                                               llm=llm,
                                               embed_model=embed_model, 
                                               show_progress=True)
vector_index.storage_context.persist(persist_dir=persist_dir)
query_engine = vector_index.as_query_engine()

prompt = "You have been given a documents that describe the firm's investment strategy. Highlight portions of the document that include a unique opportunity to invest in pre-IPO companies and put investors on a level playing field with VCs and big institutional investors. You need to use this information to create a marketing brochure that will attract investors to the firm's products. The brochure should be informative, engaging, and persuasive. You must disclose in fine print that there are no guarantees, and investing is risky so nothing promissory."
response = query_engine.query(prompt)
print(response.response)

from llama_index.core import get_response_synthesizer
from llama_index.core import DocumentSummaryIndex
from llama_index.core.node_parser import SentenceSplitter

# splitter = SentenceSplitter(chunk_size=4000, chunk_overlap=100)
response_synthesizer = get_response_synthesizer(
    response_mode="tree_summarize", use_async=True
)
doc_summary_index = DocumentSummaryIndex.from_documents(
    documents,
    llm=llm,
    # transformations=[splitter],
    response_synthesizer=response_synthesizer,
    show_progress=True,
)


query_engine = doc_summary_index.as_query_engine(
    response_mode="tree_summarize", use_async=True, top_k=5
)

prompt = "You are a marketing agent for financial firms. From the given context, create a marketing letter that highlights portions of the context including a unique opportunity to invest in pre-IPO companies and put investors on a level playing field with VCs and big institutional investors, qualifications of the investment team, and why they should invest now. Use exerpts from the context. At the end of the letter, you must disclose in fine print that there are no guarantees, and investing is risky."
response = query_engine.query(prompt)
print(response.response)

for node in response.source_nodes:
    print(node.text)
    print("-"*20)

In [ ]:
from llama_index.core.response_synthesizers import TreeSummarize
summarizer = TreeSummarize(verbose=True)


In [ ]:
prompt_summary = "Use specific details from the given documents to create a professional marketing letter that highlights portions of the context including a unique opportunity to invest in pre-IPO companies and put investors on a level playing field with VCs and big institutional investors, qualifications of the investment team, and why they should invest now. Use details from the documents, but do not make up information. Be professional and persuasive. At the end of the letter, you must disclose in fine print that there are no guarantees, and investing is risky."
response = await summarizer.aget_response(prompt_summary, [doc.text for doc in documents[1:]])

In [ ]:
print(response)

In [ ]:
for node in response.source_nodes:
    print(node.text)